# Задание 2. ML

## Описание задачи
Необходимо обучить нейросеть, способную по входному изображению лица
определять пол человека на изображении.

## Описание решения
### Описание процесса подготовки данных
1. Загрузка данных с диска
2. Конвертация 3-х канального изображения в одноканальное (GRAYSCALE)
3. Изменение размеров изображений до стандартного 80 на 110
4. Создание вектора ответов, где 0 - female, 1 - male
5. Обхединение данных из папок male и female в один dataset
6. Преобразование картинки в вектор признаков, где каждый пиксель будет являться признаком
6. Нормализация признаков объектов
7. Перемешивание данных
8. Разбиение на обучающую и тестовую выборку в соотношении 70:30

### Описание используемой нейронной сети
В архитектуре финальной нейронной сети используется 2 сверточных слоя, а также GlobalAveragePooling2D. На выходе поставлена функция активации softmax, для предсказания вероятности принадлежности к каждому из классов.

### Описание параметров обучения
Количество epoch было подобрано исходя из сходимости нейронной сети и во избежание переобучения при большем значении epoch и недообучения при меньшем

### Полученные результаты
- Результат работы нейронной сети на обучающей выборке:  90.84298795748519
- Результат работы нейронной сети на тестовой выборке:  88.680377321352

### Инструкция по запуску тренировки
Последовательно запустить все ячейки, предварительно разкоментировав данные строки:

#model.compile('adam', 'categorical_crossentropy', ['accuracy'])
#history = model.fit(X_train, y_train, epochs=15)

Данные необходимо разместить в папке с файлом notebook соответственно в internship_data/female и в internship_data/male

### Инструкция по запуску нейронной сети
python task_2.py [папка с картинками с расширением .jpg]

In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from sklearn.utils import shuffle
from keras.utils import np_utils


import cv2, glob

global inputShape, size


Using TensorFlow backend.


In [35]:
def kerasModel4():
        model = Sequential()
        model.add(Conv2D(16, (8, 8), strides=(4, 4), padding='valid', input_shape=(sizeW,sizeH,1)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (5, 5), padding="same"))
        model.add(Activation('relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Dense(512))
        model.add(Dropout(.1))
        model.add(Activation('relu'))
        model.add(Dense(2))
        model.add(Activation('softmax'))
        return model

In [23]:
sizeW = 80
sizeH = 110

# load data from female folder
femaleImages = glob.glob("internship_data/female/*.jpg")

female_data_img = [cv2.imread(img, 0) for img in femaleImages]
for i in range(0, len(female_data_img)):
    female_data_img[i] = cv2.resize(female_data_img[i],(sizeW, sizeH))
female_data = np.asarray(female_data_img)


# load data from male folder
maleImages = glob.glob("internship_data/male/*.jpg")

male_data_img = [cv2.imread(img, 0) for img in maleImages]
for i in range(0, len(male_data_img)):
    male_data_img[i] = cv2.resize(male_data_img[i],(sizeW, sizeH))
male_data = np.asarray(male_data_img)

In [27]:
from sklearn.model_selection import train_test_split


# Формирование вектора ответов
y_female = np.zeros([female_data.shape[0]],dtype = int)
y_male = np.ones([male_data.shape[0]],dtype = int)

y_data = []

y_data.extend(y_female)
y_data.extend(y_male)
y_data = np.asarray(y_data)

y_data = np_utils.to_categorical(y_data)

# Формирование датасета
X_data = []

X_data.extend(female_data)
X_data.extend(male_data)
X_data = np.asarray(X_data)

X_data = X_data.reshape(X_data.shape[0], sizeW, sizeH, 1)

for data_point in X_data:
    data_point = data_point / 255

# Перемешивание данных
X_data, y_data = shuffle(X_data, y_data)

# Разделение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0)

In [28]:
print("Train shape X", X_train.shape)
print("Train shape y", y_train.shape)
print("Test shape X", X_test.shape)
print("Test shape y", y_test.shape)

Train shape X (70001, 80, 110, 1)
Train shape y (70001, 2)
Test shape X (30001, 80, 110, 1)
Test shape y (30001, 2)


In [32]:
pd.DataFrame(y_train)[0].value_counts()

1.0    35139
0.0    34862
Name: 0, dtype: int64

In [33]:
pd.DataFrame(y_test)[0].value_counts()

0.0    15139
1.0    14862
Name: 0, dtype: int64

In [36]:
inputShape = (sizeW, sizeH, 1)
model = kerasModel4()

#model.compile('adam', 'categorical_crossentropy', ['accuracy'])
#history = model.fit(X_train, y_train, epochs=15)
model = load_model('model.h5')

metricsTrain = model.evaluate(X_train, y_train)
print("Training Accuracy: ",metricsTrain[1]*100,"%")

metricsTest = model.evaluate(X_test,y_test)
print("Testing Accuracy: ",metricsTest[1]*100,"%")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
70001/70001 [==============================] - 131s 2ms/step - loss: 0.5866 - acc: 0.6857
Epoch 2/15
70001/70001 [==============================] - 126s 2ms/step - loss: 0.4544 - acc: 0.7852
Epoch 3/15
70001/70001 [==============================] - 125s 2ms/step - loss: 0.3920 - acc: 0.8213
Epoch 4/15
70001/70001 [==============================] - 125s 2ms/step - loss: 0.3544 - acc: 0.8412
Epoch 5/15
70001/70001 [==============================] - 123s 2ms/step - loss: 0.3325 - acc: 0.8523
Epoch 6/15
70001/70001 [==============================] - 122s 2ms/step - loss: 0.3087 - acc: 0.8644
Epoch 7/15
70001/70001 [==============================] - 122s 2ms/step - loss: 0.2947 - acc: 0.8715
Epoch 8/15
70001/70001 [==============================] - 

In [37]:
model.save('model.h5')
print("Saved model to disk")

Saved model to disk
